In [1]:
from qiskit import providers,execute,QuantumRegister,ClassicalRegister,QuantumCircuit,transpile,Aer,assemble,IBMQ
from qiskit.tools import job_monitor
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import plot_histogram,plot_bloch_multivector
from numpy.random import randint,random_integers
import numpy as np
import math

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def encode_message(bits,bases):
    message=[]
    for i in range(n):
        qc=QuantumCircuit(1,1)
        if(bases[i]==1):#z basis
            if(bits[i]==0):
                pass
            else:
                qc.x(0)
        else:#x basis
            if(bits[i]==0):
                qc.h(0)#creating |+> state 
            else:
                qc.x(0)#converting to 1 
                qc.h(0)#creating |-> state
        message.append(qc)
    return message
  
def measure_message(message,bases): 
    measured=[]
    for j in range(n):
        if(bases[j]==1):# z bases
            message[j].measure(0,0)
        else:# x bases
            message[j].h(0)
            message[j].measure(0,0)
        simul=Aer.get_backend('aer_simulator')
        qobj=assemble(message[j],shots=8192,memory=True)
        result=simul.run(qobj).result()
        measurement=int(result.get_memory()[0])
        measured.append(measurement)
    return measured

def key_selection(s_bases,r_bases,bits):
    good_bits=[]  
    for i in range(n):
        if(s_bases[i]==r_bases[i]):
            good_bits.append(bits[i])
    return good_bits

def sampling(bits,string):
    sample=[]
    for i in string:
        i=np.mod(i,len(bits))#ensuring that the sample string doesnot exceed the key
        sample.append(bits.pop(i))
    return sample



def enhanced_key():
    enhanced_key=[]
    enhanced_key_r=[]
    for i in range(63):
        sender_bits=np.random.randint(2,size=n)
        sender_basis=randint(2,size=n)
        reciever_basis=randint(2,size=n)
        data=encode_message(sender_bits,sender_basis)
        key=measure_message(data,reciever_basis)
        s_key=key_selection(sender_basis,reciever_basis,sender_bits)
        for j in range(len(s_key)):
            enhanced_key.append(s_key[j])
        r_key=key_selection(sender_basis,reciever_basis,key)
        for j in range(len(r_key)):
            enhanced_key_r.append(r_key[j])
        
    return enhanced_key,enhanced_key_r

def enhanced_key_eve():
    enhanced_key=[]
    enhanced_key_r=[]
    for i in range(63):
        sender_bits=np.random.randint(2,size=n)
        sender_basis=randint(2,size=n)
        attacker_basis=randint(2,size=n)
        reciever_basis=randint(2,size=n)
        data=encode_message(sender_bits,sender_basis)
        intercepted_message = measure_message(data,attacker_basis)
        key=measure_message(data,reciever_basis)
        s_key=key_selection(sender_basis,reciever_basis,sender_bits)
        for j in range(len(s_key)):
            enhanced_key.append(s_key[j])
        r_key=key_selection(sender_basis,reciever_basis,key)
        for j in range(len(r_key)):
            enhanced_key_r.append(r_key[j])
        
    return enhanced_key,enhanced_key_r

In [3]:
###Ideal Condition without eve in noisless enviornment
from qiskit.circuit import quantumcircuit
#np.random.seed(seed=0)
n=32
s_key,r_key=enhanced_key()
print("The length of the shifted key is:")
print(len(s_key))


s=math.ceil(len(s_key) * 0.25)
print(" sample .... ", s)
sample_size=randint(0,s)

print("length of sifted key is ",len(s_key) )
print("length of sample taken is ",sample_size)
sample_string = randint(n, size=sample_size)
sender_sample=sampling(s_key,sample_string)
reciever_sample=sampling(r_key,sample_string)

mismatch = 0
for i in range(sample_size):
    if(sender_sample[i]!=reciever_sample[i]):
        mismatch = mismatch+1
print("Length of the sender sample is")
print(len(sender_sample))
print("Length of the receiver sample is")
print(len(reciever_sample))


if sender_sample==reciever_sample:
    print("Cryptographic_key_is length is")
    print(len(s_key))
else:
    print("error or Eve's Dropping")
    error_rate=mismatch/sample_size
    print("Error_rate=",error_rate)
        
    

The length of the shifted key is:
997
 sample ....  250
length of sifted key is  997
length of sample taken is  190
Length of the sender sample is
190
Length of the receiver sample is
190
Cryptographic_key_is length is
807


In [4]:
###Ideal Condition without eve in noisless enviornment
from qiskit.circuit import quantumcircuit
#np.random.seed(seed=0)
n=32
s_key,r_key=enhanced_key_eve()
print("The length of the shifted key is:")
print(len(s_key))


s=math.ceil(len(s_key) * 0.25)
print(" sample .... ", s)
sample_size=randint(0,s)

print("length of sifted key is ",len(s_key) )
print("length of sample taken is ",sample_size)
sample_string = randint(n, size=sample_size)
sender_sample=sampling(s_key,sample_string)
reciever_sample=sampling(r_key,sample_string)

mismatch = 0
for i in range(sample_size):
    if(sender_sample[i]!=reciever_sample[i]):
        mismatch = mismatch+1
print("Length of the sender sample is")
print(len(sender_sample))
print("Length of the receiver sample is")
print(len(reciever_sample))


if sender_sample==reciever_sample:
    print("Cryptographic_key_is length is")
    print(len(s_key))
else:
    print("Error")
    error_rate=mismatch/sample_size
    print("Error_rate=",error_rate)
        
    

The length of the shifted key is:
997
 sample ....  250
length of sifted key is  997
length of sample taken is  203
Length of the sender sample is
203
Length of the receiver sample is
203
Error
Error_rate= 0.3793103448275862
